In [13]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import math
import ipywidgets
import numpy as np
import matplotlib.pyplot as plt
from gathering_data_function import gathering_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
decision_matrix, data_filename, weights, normalized_weights, beneficial_criteria, non_beneficial_criteria = gathering_data(
    'C:/Users/Virgi/OneDrive/Bureau/MODM_tool_project/TOPSIS/data_input/mock_data.csv')

              Price ($)  Storage (GB)  Camera (MP)  Looks (1-5)
Alternative                                                    
Phone 1             250            16           12            5
Phone 2             200            16            8            3
Phone 3             300            32           16            4
Phone 4             275            32            8            4
Phone 5             225            16           16            2


## **Min-Max Normalization**
### *Beneficial Criteria*
For each beneficial criterion:


$
x^{normalized}_{i, j}=\frac{x_{i, j}-min(x_j)}{max(x_j)-min(x_j)}
$
### *Non-Beneficial Criteria*
For each non-beneficial criterion:


$
x^{normalized}_{i, j}=\frac{max(x_j)-x_{i, j}}{max(x_j)-min(x_j)}
$

In [37]:
def min_max_normalization(decision_matrix, beneficial_criteria):
    """
    Normalize the decision matrix using min-max normalization.
    
    Parameters:
    - decision_matrix: DataFrame containing the raw scores for each alternative and criterion.
    
    Returns:
    - normalized_matrix: DataFrame containing the normalized scores.
    """
    normalized_matrix = (decision_matrix - decision_matrix.min()) / (decision_matrix.max() - decision_matrix.min())
    for criterion in decision_matrix.columns:
        if criterion not in beneficial_criteria:
            normalized_matrix[criterion] = 1 - normalized_matrix[criterion]
    return normalized_matrix

normalized_matrix = min_max_normalization(decision_matrix, beneficial_criteria)
print(normalized_matrix)

              Price ($)  Storage (GB)  Camera (MP)  Looks (1-5)
Alternative                                                    
Phone 1            0.50           0.0          0.5     1.000000
Phone 2            1.00           0.0          0.0     0.333333
Phone 3            0.00           1.0          1.0     0.666667
Phone 4            0.25           1.0          0.0     0.666667
Phone 5            0.75           0.0          1.0     0.000000


## **Preference Functions**
### *Usual Preference Function:*  
$
P(d) = 
\begin{cases} 
0 & \text{if } d \leq 0 \\
1 & \text{if } d > 0 
\end{cases}
$
### *Linear Preference Function:*
$
P(d) = 
\begin{cases} 
0 & \text{if } d \leq q \\
\frac{d - q}{p - q} & \text{if } q < d \leq p \\
1 & \text{if } d > p 
\end{cases}
$
### *Gaussian Preference Function:*
$
P(d) = 1 - e^{-\frac{d^2}{2s^2}}
$


In [56]:
def usual_preference_function(d):
    """
    Usual Preference Function.
    
    Parameters:
    - d (float): Difference between evaluations of two alternatives for a criterion.
    
    Returns:
    - float: Preference value.
    """
    if d <= 0:
        return 0
    else:
        return 1


def linear_preference_function(d, q, p):
    """
    Linear Preference Function.
    
    Parameters:
    - d (float): Difference between evaluations of two alternatives for a criterion.
    - q (float): Indifference threshold.
    - p (float): Strict preference threshold.
    
    Returns:
    - float: Preference value.
    """
    if d <= q:
        return 0
    elif q < d <= p:
        return (d - q) / (p - q)
    else:
        return 1


def gaussian_preference_function(d, s):
    """
    Gaussian Preference Function.
    
    Parameters:
    - d (float): Difference between evaluations of two alternatives for a criterion.
    - s (float): Standard deviation of the Gaussian function.
    
    Returns:
    - float: Preference value.
    """
    if d <= 0:
        return 0
    return 1 - math.exp(-d**2 / (2 * s**2))

In [26]:
def plot_gaussian_preference_function(s):
    X = np.arange(0, 1, 0.01)
    Y = 1 - np.exp(-X**2 / (2 * s**2))
    plt.plot(X, Y)
ipywidgets.interact(plot_gaussian_preference_function, s=(0.01, 2, 0.1))

interactive(children=(FloatSlider(value=0.91, description='s', max=2.0, min=0.01), Output()), _dom_classes=('w…

<function __main__.plot_gaussian_preference_function(s)>

The parameter $d$ represents the difference in the evaluation of two alternatives with respect to a specific criterion.

$d(a_i, a_j) = f(a_i) - f(a_j)$

Where:

$a_i$ and $a_j$ are two alternatives being compared.


$f(a_i)$ and $f(a_j)$ are the evaluations of the alternatives $a_i$ and $a_j$ respectively, for a specific criterion.



In [38]:
def compute_d_values(decision_matrix):
    """
    Computes the pairwise differences for each criterion and each pair of alternatives.
    
    Parameters:
    - decision_matrix: A pandas DataFrame containing the normalized decision matrix.
    
    Returns:
    - A pandas DataFrame containing the pairwise differences.
    """
    alternatives = decision_matrix.index
    criteria = decision_matrix.columns
    
    # Create an empty dataframe to store the results
    d_values_df = pd.DataFrame()
    
    # Iterate over each criterion
    for criterion in criteria:
        # Compute pairwise differences for the current criterion
        for alt_i in alternatives:
            for alt_j in alternatives:
                if alt_i != alt_j:
                    d_key = f"d({alt_i}, {alt_j})"
                    d = decision_matrix.loc[alt_i, criterion] - decision_matrix.loc[alt_j, criterion]
                    d_values_df.loc[d_key, criterion] = d
                    
    return d_values_df


d_values_df = compute_d_values(normalized_matrix)
print(d_values_df)

                      Price ($)  Storage (GB)  Camera (MP)  Looks (1-5)
d(Phone 1, Phone 2)       -0.50           0.0          0.5     0.666667
d(Phone 1, Phone 3)        0.50          -1.0         -0.5     0.333333
d(Phone 1, Phone 4)        0.25          -1.0          0.5     0.333333
d(Phone 1, Phone 5)       -0.25           0.0         -0.5     1.000000
d(Phone 2, Phone 1)        0.50           0.0         -0.5    -0.666667
d(Phone 2, Phone 3)        1.00          -1.0         -1.0    -0.333333
d(Phone 2, Phone 4)        0.75          -1.0          0.0    -0.333333
d(Phone 2, Phone 5)        0.25           0.0         -1.0     0.333333
d(Phone 3, Phone 1)       -0.50           1.0          0.5    -0.333333
d(Phone 3, Phone 2)       -1.00           1.0          1.0     0.333333
d(Phone 3, Phone 4)       -0.25           0.0          1.0     0.000000
d(Phone 3, Phone 5)       -0.75           1.0          0.0     0.666667
d(Phone 4, Phone 1)       -0.25           1.0         -0.5    -0

In [59]:
def compute_preference_values(d_values_df, preference_functions):
    """
    Compute the preference values for each criterion and each pair of alternatives.
    
    Parameters:
    - differences_df: DataFrame containing the pairwise differences.
    - preference_functions: Dictionary mapping criteria to their corresponding preference functions.
    
    Returns:
    - DataFrame containing the preference values.
    """
    preference_values_df = d_values_df.copy()
    preference_values_df.index = ['P(' + str(idx) + ')'for idx in preference_values_df.index]


    for criterion, pref_func in preference_functions.items():
        preference_values_df[criterion] = preference_values_df[criterion].apply(pref_func)
    
    return preference_values_df


def define_preference_functions(decision_matrix):
    preference_functions = {}
    for criterion in decision_matrix.columns:
        response = input(f"What preference function do you want to use for {criterion}?\nEnter 'u' for usual preference function.\nEnter 'l' for linear preference function.\nEnter 'g' for gaussian preference function.\n").lower()
        while response not in ['u', 'l', 'g']:
            print(
                "Invalid input. Please enter 'u', 'l' or 'g'.")
            response = input(
                f"What preference function do you want to use for {criterion}? ").lower()
        if response == 'u':
            preference_functions[criterion] = lambda d: usual_preference_function(d)   
        elif response == 'l':
            preference_functions[criterion] = lambda d: linear_preference_function(d, q=0.2, p=0.8) 
        else:
            preference_functions[criterion] = lambda d: gaussian_preference_function(d, s=0.3)
    return preference_functions


preference_functions = define_preference_functions(decision_matrix)
preference_values_df = compute_preference_values(d_values_df, preference_functions)
print(preference_values_df)

                         Price ($)  Storage (GB)  Camera (MP)  Looks (1-5)
P(d(Phone 1, Phone 2))    0.000000             0     0.750648     0.915342
P(d(Phone 1, Phone 3))    0.500000             0     0.000000     0.460592
P(d(Phone 1, Phone 4))    0.083333             0     0.750648     0.460592
P(d(Phone 1, Phone 5))    0.000000             0     0.000000     0.996134
P(d(Phone 2, Phone 1))    0.500000             0     0.000000     0.000000
P(d(Phone 2, Phone 3))    1.000000             0     0.000000     0.000000
P(d(Phone 2, Phone 4))    0.916667             0     0.000000     0.000000
P(d(Phone 2, Phone 5))    0.083333             0     0.000000     0.460592
P(d(Phone 3, Phone 1))    0.000000             1     0.750648     0.000000
P(d(Phone 3, Phone 2))    0.000000             1     0.996134     0.460592
P(d(Phone 3, Phone 4))    0.000000             0     0.996134     0.000000
P(d(Phone 3, Phone 5))    0.000000             1     0.000000     0.915342
P(d(Phone 4, Phone 1))   